# Stacked Diagnostics

- Gives us additional diagnostic information when exception occurs
- Gives us access to error stacks 
- More overhead
    - Do not use if the problem can be pinned down easily
- example:
```
DO $$
    DECLARE
    some_var_1 type;
    some_var_2 type;
    some_var_3 type;
    BEGIN
        <query....>;
        <query....>;
    EXCEPTION
        WHEN others THEN
        GET STACKED DIAGNOSTICS 
            some_var_1 = MESSAGE_TEXT,
            some_var_2 = PG_EXCEPTION_DETAIL
            some_var_3 = PG_EXCEPTION_CONTEXT;
        <query....>;
        RAISE INFO 'Exception Messaage: % | Exception Details: %', some_var_1, some_var_2;
END$$;
```


<center><img src="images/04.01.jpg"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/04.02.jpg"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/04.04.jpg"  style="width: 400px, height: 300px;"/></center>


# Debugging Function

```
CREATE OR REPLACE FUNCTION func_name(
    sql_stmt TEXT
) RETURNS BOOLEAN AS 
$$
    DECLARE
        var1 TEXT;
        var2 TEXT;
        var3 TEXT;
        var4 TEXT;
    BEGIN
        BEGIN
            EXECUTE sql_stmt;
            EXCEPTION 
                WHEN others THEN
                GET STACKED DIAGNOSTICS
                    var1 = RETURNED_SQLSTATE,
                    var2 = MESSAGE_TEXT,
                    var3 = PG_EXCEPTION_DETAIL,
                    var4 = PG_EXCEPTION_CONTEXT;
            INSERT into errors (msg, state, detail, context) values (var1, var2, var3, var4);
        RETURN True; -- Exception caught
        END;
    RETURN False; -- No exception caught
    END;
$$ LANGUAGE plpgsql;
```

# Executing the Debugging Function

### Way 01

- Notice the double quote after `WHERE`
```
SELECT func_name(
'UPDATE table_name SET cost = 35.0 WHERE name = ''Macaron'' '
);
SELECT * FROM errors;
```

### Way 02

- Notice the double quote after `WHERE`
- Most desirable way of executing Debugging Function
```
DO $$
    DECLARE
        stmt VARCHAR(100) := 'UPDATE table_name SET cost = 35.0 WHERE name = ''Macaron'' ';
    BEGIN
        EXECUTE stmt;
        EXCEPTION WHEN OTHERS THEN
        PERFORM func_name(stmt);
END; $$ language 'plpgsql';
```